In [1]:
import numpy as np 
import pandas as pd 
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
dataEarthquakes = pd.read_csv('https://raw.githubusercontent.com/benvictoria17/DataAnalytics/master/dataset/Global%20Significant%20Earthquake%20Database%20from%202150BC/Worldwide-Earthquake-database.csv')
dataEarthquakes = dataEarthquakes.replace('    ', np.nan) 
dataEarthquakes.head()

,I_D,FLAG_TSUNAMI,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,FOCAL_DEPTH,EQ_PRIMARY,...,TOTAL_MISSING,TOTAL_MISSING_DESCRIPTION,TOTAL_INJURIES,TOTAL_INJURIES_DESCRIPTION,TOTAL_DAMAGE_MILLIONS_DOLLARS,TOTAL_DAMAGE_DESCRIPTION,TOTAL_HOUSES_DESTROYED,TOTAL_HOUSES_DESTROYED_DESCRIPTION,TOTAL_HOUSES_DAMAGED,TOTAL_HOUSES_DAMAGED_DESCRIPTION
0,1,No,-2150,NaN,NaN,NaN,NaN,NaN,NaN,7.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Yes,-2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,No,-2000,NaN,NaN,NaN,NaN,NaN,18.0,7.1,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN
3,5877,Yes,-1610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN
4,8,No,-1566,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
earthquakeOccurence = dataEarthquakes[['YEAR','I_D']].groupby('YEAR').count().reset_index()
earthquakeOccurence['percent'] = round(earthquakeOccurence['I_D']/earthquakeOccurence['I_D'].sum()*100,2)
earthquakeOccurence['cumPercent'] = earthquakeOccurence['percent'].cumsum()

In [4]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=earthquakeOccurence.YEAR, y=earthquakeOccurence.cumPercent, line=dict(color='rgb(33,125,196)')))
fig.update_layout(plot_bgcolor='white',
                  xaxis=dict(title='Year',ticks="outside", tickwidth=2, tickcolor='grey', ticklen=10,showline=True, linewidth=2, linecolor='grey'),
                  yaxis=dict(title='Percentile [%ile]',ticks="outside", tickwidth=2, tickcolor='grey', ticklen=10,showline=True, linewidth=2, linecolor='grey'),
                  title=dict(text="Cumulative distribution of earthquake events per year", font=dict(family="Verdana",size=25,color="Black")))
fig.update_xaxes(tickfont=dict(family='Verdana', color='grey', size=14), titlefont=dict(family='Verdana', color='black', size=16))
fig.update_yaxes(tickfont=dict(family='Verdana', color='grey', size=14), titlefont=dict(family='Verdana', color='black', size=16))
fig.show()

In [5]:
missingValues = pd.DataFrame(dataEarthquakes['YEAR'])
missingValues['countMissing'] = dataEarthquakes.isna().sum(axis=1)
missingValues = missingValues[['YEAR','countMissing']].groupby('YEAR').sum().reset_index()
missingValues = missingValues.merge(earthquakeOccurence[['YEAR','I_D']], on='YEAR')
missingValues['countMissingPercent'] = round(missingValues['countMissing']/(missingValues['I_D']*47)*100,2)

In [6]:
fig = go.Figure()
fig.add_trace(go.Bar(x=missingValues.YEAR, y=missingValues.countMissingPercent))
fig.update_layout(plot_bgcolor='white',
                  xaxis=dict(title='Year',ticks="outside", tickwidth=2, tickcolor='grey', ticklen=10,showline=True, linewidth=2, linecolor='grey'),
                  yaxis=dict(title='Percentage of missing data [%]',ticks="outside", tickwidth=2, tickcolor='grey', ticklen=10,showline=True, linewidth=2, linecolor='grey'),
                  title=dict(text="Percentage of missing data per year", font=dict(family="Verdana",size=25,color="Black")))
fig.update_xaxes(tickfont=dict(family='Verdana', color='grey', size=14), titlefont=dict(family='Verdana', color='black', size=16))
fig.update_yaxes(tickfont=dict(family='Verdana', color='grey', size=14), titlefont=dict(family='Verdana', color='black', size=16))
fig.show()

In [7]:
fig = go.Figure()
fig.add_trace(go.Bar(x=[(dataEarthquakes['FLAG_TSUNAMI'] == 'Yes').sum(),(dataEarthquakes['FLAG_TSUNAMI'] == 'No').sum()],
                     y=['Yes','No'],
                     text=[str(round((dataEarthquakes['FLAG_TSUNAMI'] == 'Yes').sum()/len(dataEarthquakes)*100,2))+'%',str(round((dataEarthquakes['FLAG_TSUNAMI'] == 'No').sum()/len(dataEarthquakes)*100,2))+'%'],
                     textposition='inside',
                     orientation='h',
                     marker=dict(
        color='rgb(33,125,196)')))
fig.update_layout(plot_bgcolor='white',
                  xaxis=dict(showticklabels=False),
                  title=dict(text="Earthquake triggering a tsunami?", font=dict(family="Verdana",size=25,color="Black")),
                  font=dict(family="Verdana",
                            size=18,
                            color="grey"),
                 margin=dict(b=10),
                 height=300, width=800)
fig.show()

In [8]:
allData = []
color = ['blue','orange']

for y in dataEarthquakes['FLAG_TSUNAMI'].unique():
    data=go.Scattermapbox(lon = dataEarthquakes['LONGITUDE'][dataEarthquakes['FLAG_TSUNAMI'] == y],
                           lat=dataEarthquakes['LATITUDE'][dataEarthquakes['FLAG_TSUNAMI'] == y],
                       text = ['Year: ' + str(list(dataEarthquakes['YEAR'][dataEarthquakes['FLAG_TSUNAMI'] == y])[x]) for x in range(len(dataEarthquakes[dataEarthquakes['FLAG_TSUNAMI'] == y]))],
                       hoverinfo = 'text',
                       mode = 'markers',
                       name= 'No tsunami triggerd' if y == 'No' else 'Tsunami triggered',
                       marker=go.scattermapbox.Marker(size=[5]*len(dataEarthquakes['LATITUDE']),
                                                      sizeref = 1,
                                                      color= ['blue']*len(dataEarthquakes['FLAG_TSUNAMI'] == y) if y == 'Yes' else ['orange']*len(dataEarthquakes['FLAG_TSUNAMI'] == y),
                                                      opacity=0.5))
    allData.append(data)

layout = go.Layout(autosize = True, showlegend=True,
                   title=dict(text="Earthquake localisation per tsunami triggered", font=dict(family="Verdana",size=25,color="Black")),
                   margin=go.layout.Margin(l=50,r=20,b=10,t=50,pad=4),
                   mapbox=go.layout.Mapbox(bearing = 0,
                                           pitch=0, 
                                           zoom=0.6,
                                           style='basic'),
                   mapbox_style="stamen-terrain",
                   width=1050,height=550)
fig = go.Figure(data = allData,layout=layout)
fig.show()

In [9]:
fig = go.Figure()
fig.add_trace(go.Densitymapbox(lat=dataEarthquakes['LATITUDE'][dataEarthquakes['EQ_PRIMARY'].dropna().index],
                               lon=dataEarthquakes['LONGITUDE'][dataEarthquakes['EQ_PRIMARY'].dropna().index],
                               z=dataEarthquakes['EQ_PRIMARY'][dataEarthquakes['EQ_PRIMARY'].dropna().index],
                               radius=5))

fig.update_layout(autosize = True, showlegend=True,
                   title=dict(text="Earthquake localisation by magnitude", font=dict(family="Verdana",size=25,color="Black")),
                   margin=go.layout.Margin(l=50,r=20,b=10,t=50,pad=4),
                   mapbox=go.layout.Mapbox(zoom=0.6,
                                           style='basic'),
                   mapbox_style="stamen-terrain",
                   width=950,height=550)
fig.show()
fig = go.Figure()
fig.add_trace(go.Densitymapbox(lat=dataEarthquakes['LATITUDE'][dataEarthquakes['FOCAL_DEPTH'].dropna().index],
                               lon=dataEarthquakes['LONGITUDE'][dataEarthquakes['FOCAL_DEPTH'].dropna().index],
                               z=dataEarthquakes['FOCAL_DEPTH'][dataEarthquakes['FOCAL_DEPTH'].dropna().index],
                               radius=5))

fig.update_layout(autosize = True, showlegend=True,
                   title=dict(text="Earthquake localisation by depth", font=dict(family="Verdana",size=25,color="Black")),
                   margin=go.layout.Margin(l=50,r=20,b=10,t=50,pad=4),
                   mapbox=go.layout.Mapbox(zoom=0.6,
                                           style='basic'),
                   mapbox_style="stamen-terrain",
                   width=950,height=550)
fig.show()

In [10]:
depthBinSize = 25 
magnitudeBinSize = 0.2
binDepth = list(range(0,700,depthBinSize))
binMagnitude = np.arange(0,10,magnitudeBinSize)
dfMagnitudeDepth = dataEarthquakes[['EQ_PRIMARY','FOCAL_DEPTH']].dropna()
dfMagnitudeDepth['DepthBin'] = np.floor(dfMagnitudeDepth['FOCAL_DEPTH']/depthBinSize)*depthBinSize
dfMagnitudeDepth = dfMagnitudeDepth.groupby(['DepthBin', pd.cut(dfMagnitudeDepth['EQ_PRIMARY'], binMagnitude)]).size().unstack()
matrixCountEvent = []
for x in range(len(dfMagnitudeDepth.index)):
    matrixCountEvent.append(dfMagnitudeDepth.iloc[x].values.tolist())
fig = go.Figure(data=go.Heatmap(x=binMagnitude,
                                y=binDepth,
                                z=matrixCountEvent,
                                colorscale=[[0,'rgb(255,255,255)'], [0.01,"rgb(170,210,255)"], [1,"rgb(0,30,250)"]]))
fig.update_layout(plot_bgcolor='white', width = 1000,
                  xaxis=dict(title='Magnitude',ticks="outside", tickwidth=2, tickcolor='grey', ticklen=10,showline=True, linewidth=2, linecolor='grey'),
                  yaxis=dict(title='Depth [km]',ticks="outside", tickwidth=2, tickcolor='grey', ticklen=10,showline=True, linewidth=2, linecolor='grey'),
                  title=dict(text="Count earthquake per depth and magnitude", font=dict(family="Verdana",size=25,color="Black")))
fig.update_xaxes(tickfont=dict(family='Verdana', color='grey', size=14), titlefont=dict(family='Verdana', color='black', size=16))
fig.update_yaxes(tickfont=dict(family='Verdana', color='grey', size=14), titlefont=dict(family='Verdana', color='black', size=16))
fig.show()

In [11]:
magnitudeBinSize = 0.2
binDamage = list(range(1,4))
binMagnitude = np.arange(0,10,magnitudeBinSize)
dfMagnitudeDamage = dataEarthquakes[['EQ_PRIMARY','DAMAGE_DESCRIPTION']].dropna()
dfMagnitudeDamage = dfMagnitudeDamage.groupby(['DAMAGE_DESCRIPTION', pd.cut(dfMagnitudeDamage['EQ_PRIMARY'], binMagnitude)]).size().unstack()
matrixCountEvent = []
for x in range(len(dfMagnitudeDamage.index)):
       matrixCountEvent.append(dfMagnitudeDamage.iloc[x].values.tolist())
fig = go.Figure(data=go.Heatmap(x=binMagnitude,
                                y=binDamage,
                                z=matrixCountEvent,
                                colorscale=[[0,'rgb(255,255,255)'], [0.01,"rgb(170,210,255)"], [1,"rgb(0,30,250)"]]))
fig.update_layout(plot_bgcolor='white', width = 1000,
                  xaxis=dict(title='Magnitude',ticks="outside", tickwidth=2, tickcolor='grey', ticklen=10,showline=True, linewidth=2, linecolor='grey'),
                  yaxis=dict(title='Damage',ticks="outside", tickwidth=2, tickcolor='grey', ticklen=10,showline=True, linewidth=2, linecolor='grey'),
                  title=dict(text="Count earthquake per damage and magnitude", font=dict(family="Verdana",size=25,color="Black")))
fig.update_xaxes(tickfont=dict(family='Verdana', color='grey', size=14), titlefont=dict(family='Verdana', color='black', size=16))
fig.update_yaxes(tickfont=dict(family='Verdana', color='grey', size=14), titlefont=dict(family='Verdana', color='black', size=16))
fig.show()

In [13]:
pcaData = dataEarthquakes[['YEAR','FLAG_TSUNAMI','FOCAL_DEPTH','EQ_PRIMARY','REGION_CODE','DAMAGE_DESCRIPTION']]
pcaData = pcaData.dropna(axis=0).reset_index(drop=True) # Remove NaN
pcaData = pcaData.replace({'FLAG_TSUNAMI' : { 'No' : 0, 'Yes' : 1}}) # Replace categorical values by numerical values

scaler = StandardScaler()
pcaData_scaled = scaler.fit_transform(pcaData)
pcaData_scaled

pca = PCA(n_components=6)
pca.fit(pcaData_scaled)
pcaData_projected = pca.transform(pcaData_scaled) # for scatter plots
fig = go.Figure()
[fig.add_trace(go.Scatter(x=[0, pca.components_[0,x]],y=[0,pca.components_[1,x]],name=pcaData.columns[x])) for x in range(len(pca.components_[0,:]))]
fig.update_layout(plot_bgcolor='white',height=600, width=600,
                  showlegend=False,
                  shapes=[dict(type="circle",xref="x",yref="y",x0=-1,y0=-1,x1=1,y1=1,line_color="LightSeaGreen",)],
                  xaxis=dict(ticks="outside", tickwidth=2, tickcolor='grey', ticklen=10,showline=True, linewidth=2, linecolor='grey'),
                  yaxis=dict(ticks="outside", tickwidth=2, tickcolor='grey', ticklen=10,showline=True, linewidth=2, linecolor='grey'),
                  title=dict(text="Principal Component Analysis", font=dict(family="Verdana",size=25,color="Black")))
fig.update_xaxes(title=dict(text='PC1', font=dict(size=18)),showgrid=True, linecolor='black', ticks='outside')
fig.update_yaxes(title=dict(text='PC2', font=dict(size=18)),showgrid=True, linecolor='black', ticks='outside')
fig.show()

In [14]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=[pcaData_projected[x][0] for x in range(len(pcaData_projected)) if pcaData['FLAG_TSUNAMI'][x]==0],y=[pcaData_projected[x][1] for x in range(len(pcaData_projected)) if pcaData['FLAG_TSUNAMI'][x]==0], name='No tsunami triggered', mode='markers'))
fig.add_trace(go.Scatter(x=[pcaData_projected[x][0] for x in range(len(pcaData_projected)) if pcaData['FLAG_TSUNAMI'][x]==1],y=[pcaData_projected[x][1] for x in range(len(pcaData_projected)) if pcaData['FLAG_TSUNAMI'][x]==1], name='Tsunami triggered', mode='markers'))
fig.update_layout(plot_bgcolor='white',height=500, width=1000, showlegend=True,
                  xaxis=dict(ticks="outside", tickwidth=2, tickcolor='grey', ticklen=10,showline=True, linewidth=2, linecolor='grey'),
                  yaxis=dict(ticks="outside", tickwidth=2, tickcolor='grey', ticklen=10,showline=True, linewidth=2, linecolor='grey'),
                  title=dict(text="Tsunami triggered with PCA", font=dict(family="Verdana",size=25,color="Black")))
fig.update_xaxes(title=dict(text='PC1', font=dict(size=18)),showgrid=True, linecolor='black', ticks='outside')
fig.update_yaxes(title=dict(text='PC2', font=dict(size=18)),showgrid=True, linecolor='black', ticks='outside')
fig.show()

In [15]:
pcaData = dataEarthquakes[['YEAR','FLAG_TSUNAMI','FOCAL_DEPTH','EQ_PRIMARY','REGION_CODE','DAMAGE_DESCRIPTION']]
pcaData = pcaData.dropna(axis=0).reset_index(drop=True) 
pcaData = pcaData.replace({'FLAG_TSUNAMI' : { 'No' : 0, 'Yes' : 1}}) 

In [16]:
scaler = StandardScaler()
pcaData_scaled = scaler.fit_transform(pcaData)
pcaData_scaled

pca = PCA(n_components=6)
pca.fit(pcaData_scaled)
pcaData_projected = pca.transform(pcaData_scaled) 
fig = go.Figure()
[fig.add_trace(go.Scatter(x=[0, pca.components_[0,x]],y=[0,pca.components_[1,x]],name=pcaData.columns[x])) for x in range(len(pca.components_[0,:]))]
fig.update_layout(plot_bgcolor='white',height=600, width=600,
                  showlegend=False,
                  shapes=[dict(type="circle",xref="x",yref="y",x0=-1,y0=-1,x1=1,y1=1,line_color="LightSeaGreen",)],
                  xaxis=dict(ticks="outside", tickwidth=2, tickcolor='grey', ticklen=10,showline=True, linewidth=2, linecolor='grey'),
                  yaxis=dict(ticks="outside", tickwidth=2, tickcolor='grey', ticklen=10,showline=True, linewidth=2, linecolor='grey'),
                  title=dict(text="Principal Component Analysis", font=dict(family="Verdana",size=25,color="Black")))
fig.update_xaxes(title=dict(text='PC1', font=dict(size=18)),showgrid=True, linecolor='black', ticks='outside')
fig.update_yaxes(title=dict(text='PC2', font=dict(size=18)),showgrid=True, linecolor='black', ticks='outside')
fig.show()

In [17]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=[pcaData_projected[x][0] for x in range(len(pcaData_projected)) if pcaData['FLAG_TSUNAMI'][x]==0],y=[pcaData_projected[x][1] for x in range(len(pcaData_projected)) if pcaData['FLAG_TSUNAMI'][x]==0], name='No tsunami triggered', mode='markers'))
fig.add_trace(go.Scatter(x=[pcaData_projected[x][0] for x in range(len(pcaData_projected)) if pcaData['FLAG_TSUNAMI'][x]==1],y=[pcaData_projected[x][1] for x in range(len(pcaData_projected)) if pcaData['FLAG_TSUNAMI'][x]==1], name='Tsunami triggered', mode='markers'))
fig.update_layout(plot_bgcolor='white',height=500, width=1000, showlegend=True,
                  xaxis=dict(ticks="outside", tickwidth=2, tickcolor='grey', ticklen=10,showline=True, linewidth=2, linecolor='grey'),
                  yaxis=dict(ticks="outside", tickwidth=2, tickcolor='grey', ticklen=10,showline=True, linewidth=2, linecolor='grey'),
                  title=dict(text="Tsunami triggered with PCA", font=dict(family="Verdana",size=25,color="Black")))
fig.update_xaxes(title=dict(text='PC1', font=dict(size=18)),showgrid=True, linecolor='black', ticks='outside')
fig.update_yaxes(title=dict(text='PC2', font=dict(size=18)),showgrid=True, linecolor='black', ticks='outside')
fig.show()